In [1]:
import subprocess
import importlib

# Function to install a package only if not already installed
def install_package(package_name):
    if importlib.util.find_spec(package_name) is None:
        subprocess.check_call(['pip', 'install', package_name], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Install packages if not found
install_package('import-ipynb')
install_package('gspread')
install_package('oauth2client')
install_package('google-api-python-client')
install_package('utm')

In [2]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

import import_ipynb
import contextlib
import os
import importlib
import yaml

with contextlib.redirect_stdout(open(os.devnull, "w")), contextlib.redirect_stderr(open(os.devnull, "w")):
    RFQ_Load = importlib.import_module("RFQ_Load")

class Settings:
    """A class to load and access YAML settings with attribute-style access."""
    
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            # Load YAML data and set it as class attributes
            data = yaml.safe_load(file)
            for key, value in data.items():
                setattr(self, key, value)

# Usage example:
def load_settings():
    """Load settings from settings.yaml and return a Settings object."""
    return Settings('ps_settings.yaml')

# Load the settings
ps_settings = load_settings()

# Automation algo

In [3]:
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('client_secret.json', scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=creds)

def read_google_doc(doc_id):
    """Read content from a Google Doc."""
    doc = docs_service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    
    # Extract text from the document content
    text = ''
    for element in content:
        if 'paragraph' in element:
            for paragraph_element in element['paragraph']['elements']:
                if 'textRun' in paragraph_element:
                    text += paragraph_element['textRun']['content']
    return text

def update_google_doc_multiple_replacements(doc_id, replacements):
    """Replace multiple text items in a Google Doc."""
    requests = [
        {
            'replaceAllText': {
                'containsText': {
                    'text': search_text,
                    'matchCase': True
                },
                'replaceText': replacement_text
            }
        }
        for search_text, replacement_text in replacements.items()
    ]
    
    #batch update
    result = docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
    return result

# Weather & Opencage API requests

ref:
> https://opencagedata.com/
> 
> https://openweathermap.org/api
> 
> https://power.larc.nasa.gov/data-access-viewer/

In [4]:
import requests
import utm

# Function to get latitude, longitude, and locality data from OpenCage API
def get_location_data(city, opencage_key):
    geocode_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        'q': city,
        'key': opencage_key,
        'no_annotations': 1  # Focus on essential details
    }
    
    response = requests.get(geocode_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]
            return {
                'latitude': location['geometry']['lat'],
                'longitude': location['geometry']['lng'],
                'locality': location['components'].get('city', city)
            }
    print("Failed to retrieve geolocation data.")
    return None

# Function to convert latitude and longitude to UTM coordinates
def convert_to_utm(lat, lon):
    try:
        utm_result = utm.from_latlon(lat, lon)
        
        return {
            'utm_zone': utm_result[2],
            'utm_letter': utm_result[3],
            'northing': utm_result[1],
            'easting': utm_result[0]
        }
        
    except Exception as e:
        print(f"Failed to convert to UTM: {e}")
        return None

# Function to get elevation from Open Elevation API
def get_elevation(lat, lon):
    elevation_url = "https://api.open-elevation.com/api/v1/lookup"
    params = {'locations': f"{lat},{lon}"}
    
    response = requests.get(elevation_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['results'][0]['elevation'] if 'results' in data else None
    print("Failed to retrieve elevation data.")
    return None

def get_humidity(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200:
        humidity = data['main']['humidity']
        return humidity
        #print(f"Relative Humidity in {city}: {humidity}%")
    else:
        print("Error retrieving data")

def get_simple_wind_conditions(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extracting wind conditions
        wind_speed = data['wind']['speed']  # Wind speed in meters per second
        return wind_speed*3.6
    else:
        print("Failed to retrieve wind data.")
        
def get_climate_data(lat, lon):
    # URL for NASA POWER API
    climate_url = f"https://power.larc.nasa.gov/api/temporal/climatology/point"
    params = {
        'parameters': 'T2M',  # T2M represents the 2-meter air temperature
        'community': 'RE',  # RE community is for general use
        'longitude': lon,
        'latitude': lat,
        'format': 'json'
    }
    
    response = requests.get(climate_url, params=params)
    if response.status_code == 200:
        data = response.json()
        
        if 'properties' in data and 'parameter' in data['properties'] and 'T2M' in data['properties']['parameter']:
            temperature_data = data['properties']['parameter']['T2M']
            
            # Calculate mean summer and winter temperatures
            summer_months = ['JUN', 'JUL', 'AUG']
            winter_months = ['DEC', 'JAN', 'FEB']
            
            # Calculate mean summer temperature
            summer_temps = [temperature_data[month] for month in summer_months]
            summer_temp = sum(summer_temps) / len(summer_temps)
            
            # Calculate mean winter temperature
            winter_temps = [temperature_data[month] for month in winter_months]
            winter_temp = sum(winter_temps) / len(winter_temps)
            
            return round(summer_temp, 2), round(winter_temp, 2)
    print("Failed to retrieve climate data.")
    return None, None 

# data class

ref:
> https://www.omnicalculator.com/physics/dew-point

In [5]:
import numpy as np

class data:
    def __init__(self, city, opencage_key, weather_api_key):
        self.city = city
        self.opencage_key = opencage_key
        self.weather_api_key = weather_api_key

        self.location_data = get_location_data(city, opencage_key)
        if self.location_data:
            self.lat = self.location_data['latitude']
            self.lon = self.location_data['longitude']
            
            # Convert to UTM coordinates
            self.utm = convert_to_utm(self.lat, self.lon)

            # Get elevation data
            self.elevation = get_elevation(self.lat, self.lon)

            # Get climate data
            self.summer_temp, self.winter_temp = get_climate_data(self.lat, self.lon)

            # Get humidity data
            self.humidity = get_humidity(city, weather_api_key)

            # Get dew point data
            def get_dew_point(lat, lon, api_key):
                url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
                response = requests.get(url)

                if response.status_code == 200:
                    data = response.json()
                    temperature = data['main']['temp']
                    humidity = self.humidity
                    ###############################
                    a = 17.625
                    b = 243.04 
                    al = np.log(humidity/100) + (a*temperature/(b + temperature))
                    top = b * al
                    bot = a - al
                    dp = top/bot
                    at = temperature
                    
                    return at, dp
                else:
                    return None, None
            self.dew_point = get_dew_point(self.lat, self.lon, weather_api_key)[0]
            self.ambient = get_dew_point(self.lat, self.lon, weather_api_key)[1]

            # Get wind data
            self.wind = get_simple_wind_conditions(city, weather_api_key)
            
        else:
            self.lat = self.lon = self.utm_data = None
            self.elevation = None
            self.summer_temp = self.winter_temp = None
            self.dew_point = None
            self.humidity = None
            self.wind = None
            self.ambient = None 

# Run code to get API pulls

In [6]:
d = data(ps_settings.Locality_input, ps_settings.opencage_key, ps_settings.weather_api_key)

# Logic fill to parse algo

In [7]:
%%capture
import subprocess, importlib
if importlib.util.find_spec('import-ipynb') is None:
    !pip install import-ipynb

import import_ipynb
from RFQ_MEB import *

In [8]:
%%capture
import subprocess, importlib
if importlib.util.find_spec('import-ipynb') is None:
    !pip install import-ipynb

import import_ipynb
from RFQ_MEB import *
from RFQ_Output import *

In [9]:
feed = S[ps_settings.feed]
rsg = S[ps_settings.rsg]
csg = S[ps_settings.csg]
rsgcomp = rsg.Y
csgcomp = csg.Y

print(rsg)
def get_comp(input_choice): 
    comp_dict = rsgcomp if input_choice == 'rsg' else csgcomp

    prefix = 'rsg' if input_choice == 'rsg' else 'csg'
    
    def format_value(value):
        value_scaled = float(value) * 100
        return f"<0.1" if value_scaled < 0.1 else f"{value_scaled:.2f}"

    return {
        f'{prefix}0': format_value(comp_dict[N2]),
        f'{prefix}1': format_value(comp_dict[H2]),
        f'{prefix}2': format_value(comp_dict[H2O]),
        f'{prefix}3': format_value(comp_dict[CO2]),
        f'{prefix}4': format_value(comp_dict[CO]),
        f'{prefix}5': format_value(comp_dict[C1]),
        f'{prefix}6': format_value(comp_dict[CH4]),
        f'{prefix}7': format_value(comp_dict[C2H6]),
        f'{prefix}8': format_value(comp_dict[C2H4]),
        f'{prefix}9': format_value(comp_dict[C2H2]),
        f'{prefix}x10': format_value(comp_dict[CH4O]),
        f'{prefix}x11': format_value(comp_dict[C7H8]),
        f'{prefix}x12': format_value(comp_dict[C7H8O]),
        f'{prefix}x13': format_value(comp_dict[C10H8]),
        f'{prefix}x14': format_value(comp_dict[C16H10]),
        f'{prefix}x15': format_value(comp_dict[NH3]),
        f'{prefix}x16': format_value(comp_dict[H2S]),
        f'{prefix}x17': format_value(comp_dict[Si1])
    }

rsgcomps = get_comp('rsg')
csgcomps = get_comp('csg')

================================== 205 : Bypass gas valve outlet ===================================
 tag                                  : 205 : Bypass gas valve outlet
 name                                 : Bypass gas valve outlet
 fluid                                : raw syngas
 n                                    : 336.3 kmol/h
 m                                    : 6,200 kg/h
 T                                    : 600 °C
 rho                                  : 0.2573 kg/m³
 v                                    : 2.409×10⁴ m³/h
 Q                                    : 1.382×10⁴ MJ/h
 P                                    : 101.3 kPa
 H_f                                  : -5.043e+04 MJ/h
 HHV                                  : -9.347e+04 MJ/h
 LHV                                  : -8.538e+04 MJ/h
-------------------------------- Composition Vectors / Coordinates ---------------------------------
                      N          M          Y          X
Symbol            mol/h 

In [10]:
replacements = {
    "PROJCODE": ps_settings.PROJCODE,
    "PREPFOR": ps_settings.PREPFOR,
    "PROJNAME": ps_settings.PROJNAME,
    "DATEINPUT": ps_settings.DATEINPUT,
    ########################################
    "OP_Time": ps_settings.optime,
    "Shift_Time": ps_settings.shifttime,
    "H_Avail": ps_settings.havail,
    "feed_mech": ps_settings.feedmech,
    "HTPsa": ps_settings.HTPsa,
    "AKsa": ps_settings.AKsa,
    "SAsa": ps_settings.SAsa,
    ########################################
    "MTs": f"{d.summer_temp:.2f}",
    "MTw": f"{d.winter_temp:.2f}",
    "MaT": f"{d.ambient:.2f}",
    "HMD": f"{d.humidity:.2f}",
    "DDP": f"{d.dew_point:.2f}",
    ########################################
    "MC": f"{feed.K[Moisture].m:.2f}",
    "BD": f"{design.feed.bulk_density.m}",
    "PD": f"{design.feed.particle_density.m}",
    "CV": f"{feed.CV.m:.2f}",
    "pass20": f"{design.feed.diameter_20pc_passing.m}",
    "pass99": f"{design.feed.diameter_99pc_passing.m}",
    "flowability": f"{design.feed.flowability}",
    "repose": f"{design.feed.angle_of_repose.m}",
    "nfrfeed": f"{design.feed.nominal_feed_rate.m}",
    "vfrfeed": f"{(design.feed.nominal_feed_rate/design.feed.bulk_density).m:.2f}",
    ########################################
    "pmaxt": ps_settings.pmaxt,
    "pdest": ps_settings.pdest,
    "fibcsize": ps_settings.fibcsize,
    "bulkpdt": ps_settings.bulkpdt,
    "expfibcm": ps_settings.expfibcm,
    "DFTfibc": ps_settings.DFTfibc,
    "purecden": ps_settings.purecden,
    ########################################
    "operatingt": f"{design.pyrolysis.operating_temperature.m}",
    "syngast": f"{design.pyrolysis.syngas_temperature.m}",
    "discharget": f"{design.pyrolysis.discharge_temperature.m}",
    "dischargep": f"{design.pyrolysis.discharge_pressure.m}",
    "kilnrestime": f"{design.pyrolysis.residence_time.m}",
    "dustfrac": f"{design.pyrolysis.dust_fraction.m}",
    "recupt": f"{design.pyrolysis.burner_air_temperature.m}",
    "oxygeninexhaust": f"{design.pyrolysis.dry_oxygen.m}",
    "exhaustt": f"{design.pyrolysis.exhaust_temperature.m}",
    ########################################
    "makeupwater": f"{design.gcu.makeup_water.m}",
    "exitttemp": ps_settings.exittemp,
    "dustremoval": ps_settings.dustremoval,
    "pressuredrop": ps_settings.pressuredrop,
    ########################################
    "maxexittemp": ps_settings.maxexittemp,
    "scrubpdrop": ps_settings.scrubpdrop,
    "minrecirc": ps_settings.minrecirc,
    "remodust": f"{design.gcu.removal_efficiencies.dust.m}",
    "rtar1": f"{design.gcu.removal_efficiencies.tar1.m}",
    "rtar2": f"{design.gcu.removal_efficiencies.tar2.m}",
    "rtar3": f"{design.gcu.removal_efficiencies.tar3.m}",
    "rtar4": f"{design.gcu.removal_efficiencies.tar4.m}",
    "rchlo": f"{design.gcu.removal_efficiencies.chlorides.m}",
    "ramine": f"{design.gcu.removal_efficiencies.amines.m}",
    "rsulp": f"{design.gcu.removal_efficiencies.sulphides.m}",
    "oppH": ps_settings.oppH,
    "opORP": ps_settings.opORP,
    "lowtemp": ps_settings.lowtemp,
    "lowpdrop": ps_settings.lowpdrop,
    "cakesolids": ps_settings.cakesolids,
    "filteff": ps_settings.filteff,
    ########################################
    "shaftrate": f"{design.engines.shaft_rating.m}",
    "shafteff": f"{design.engines.shaft_efficiency.m}",
    "alteff": f"{design.engines.alternator_efficiency.m}",
    ########################################
    "sactot": f"{design.SACTO.operating_temperature.m}",
    "normalskintemp": f"{design.SACTO.skin_temperature.m}",
    "maxskintemp": f"{design.SACTO.max_skin_temperature.m}",
    "heatlossrate": f"{design.SACTO.heat_loss_rate.m}",
    "sactoresitime": f"{design.SACTO.residence_time.m}",
    "LDratio": f"{design.SACTO.LD_ratio}",
    "stacktemp": f"{design.SACTO.stack_temperature.m}",
    "sdminoxy": ps_settings.sdminoxy, 
    ########################################
    "freezeprotectcool": f"{design.chilled_water.freeze_protection}",
    "coolwaterdensity": f"{design.cooling.density.m}",
    "dewpointdT": f"{design.cooling.dew_point_deltaT.m}",
    "pavailcoolwater": f"{design.cooling.pressure.m}",
    ########################################
    "freezeprotectchill": f"{design.chilled_water.freeze_protection}",
    "chillwaterdensity": f"{design.chilled_water.density.m}",
    "chillwviscosity": f"{design.chilled_water.dynamic_viscosity.m}",
    "mintempchillwater": f"{design.chilled_water.minimum_temperature}",
    "chillwaterpavail": f"{design.chilled_water.pressure}",
    ########################################
    "rsghhv": f"{(-rsg.HHV/rsg.m).m:.2f}",
    "rsglhv": f"{(-rsg.LHV/rsg.m).m:.2f}",
    "csghhv": f"{(-csg.HHV/csg.m).m:.2f}",
    "csglhv": f"{(-csg.LHV/csg.m).m:.2f}",
    ########################################
    "appdx1": f"{design}",
}

replacements.update(rsgcomps)
replacements.update(csgcomps)


print(replacements)

{'PROJCODE': 'T3061-1-90-1', 'PREPFOR': 'AVF ENERGY', 'PROJNAME': 'HP2100 XXX TO CHAR & SYNGAS PLANT', 'DATEINPUT': 'XXX 202X', 'OP_Time': '24 h/day x 7 day/wk', 'Shift_Time': '12', 'H_Avail': '8000', 'feed_mech': 'Continuous by others', 'HTPsa': '40', 'AKsa': '40', 'SAsa': '20', 'MTs': '22.86', 'MTw': '6.07', 'MaT': '4.68', 'HMD': '74.00', 'DDP': '9.07', 'MC': '1949.32', 'BD': '400.0', 'PD': '530.0', 'CV': '16.32', 'pass20': '10', 'pass99': '50', 'flowability': 'cohesive', 'repose': '23', 'nfrfeed': '7050.0', 'vfrfeed': '17.62', 'pmaxt': '90', 'pdest': '75', 'fibcsize': '1', 'bulkpdt': '260', 'expfibcm': '260', 'DFTfibc': '5', 'purecden': '2266', 'operatingt': '850', 'syngast': '750', 'discharget': '600', 'dischargep': '1013', 'kilnrestime': '20', 'dustfrac': '2.5', 'recupt': '200', 'oxygeninexhaust': '4.0', 'exhaustt': '550', 'makeupwater': '250.0', 'exitttemp': '65', 'dustremoval': '95', 'pressuredrop': '10', 'maxexittemp': '35', 'scrubpdrop': '25', 'minrecirc': '65', 'remodust': '9

# Run sequence to update

In [11]:
DOCUMENT_ID = ps_settings.doc_id
update_result = update_google_doc_multiple_replacements(DOCUMENT_ID, replacements)